# SET ENVIRONMENT

In [ ]:
# from google.colab import drive
# drive.mount('/gdrive')

In [ ]:
# %cd /gdrive/My Drive/Colab Notebooks/AI in biomed

In [3]:
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import scipy.io
import os
from sklearn import preprocessing
import pandas as pd
from scipy import stats, signal
import pickle
import csv

In [4]:
import tensorflow as tf
from tensorflow import keras

from keras import layers, Sequential
from keras.layers import Input, Add, Dense, Activation, ZeroPadding1D, Flatten, Conv1D, MaxPooling1D, GlobalMaxPooling1D, Dropout
from keras.models import Model

import random

import keras.backend as K
K.set_image_data_format('channels_last')

# LOAD & PLOT DATA

In [5]:
seed=12

In [6]:
mat = scipy.io.loadmat('../input/aibiomed/training_set/S109_250.mat')
mat_ann = scipy.io.loadmat('../input/aibiomed/training_set/S109_250_ann.mat')
mat_rpk = scipy.io.loadmat('../input/aibiomed/training_set/S109_250_rpk.mat')

signal = mat['ecg']
annots = mat_ann['labels']
rpeaks = mat_rpk['rpeaks']

print('Shape rpeaks: {} \nShape annots: {}'.format(rpeaks.shape, annots.shape))

# First 500 samples of both leads and R peaks
plt.figure(figsize = (18,5))
plt.subplot(1,2,1)
plt.plot(signal[:1000,0])
plt.plot(rpeaks[:5], signal[rpeaks[:5],0], 'k*')
plt.title('LEAD 1')
plt.grid()
plt.subplot(1,2,2)
plt.plot(signal[:1000,1] )
plt.plot(rpeaks[:5], signal[rpeaks[:5],1], 'k*')
plt.title('LEAD 2')
plt.grid()


In [7]:
ann_types = np.unique(annots)
print('Types of labels: {}'.format(ann_types))

In [8]:
def filter_signal(signal, fs):
    N = 4
    Range = [0.5, 35] 
    Wn = [r/(fs/2) for r in Range]
    b, a = scipy.signal.butter(N, Wn, 'bandpass')
    output_signal = scipy.signal.filtfilt(b, a, signal)
    return output_signal
    

In [9]:
signal_filt_0 = filter_signal(signal[:,0], fs=250)
signal_filt_1 = filter_signal(signal[:,1], fs=250)

# First 500 samples of both leads and R peaks
plt.figure(figsize = (18,5))
plt.subplot(1,2,1)
plt.plot(signal[:1000,0])
plt.plot(signal_filt_0[:1000])
plt.title('LEAD 1')
plt.grid()
plt.subplot(1,2,2)
plt.plot(signal[:1000,1])
plt.plot(signal_filt_1[:1000])
plt.title('LEAD 2')
plt.grid()


# CREATE DATASET

This includes taking the R peak positions and splitting the signals into independet ECG beats. The window is fixed -> 35 samples before and after the R peak position. Each peak is normalized between -1 and 1. The ECG beat which is an array is elongated with some values: 
* logarithm of the current RR distance
* logarithm of the next RR distance
* logarithm of an average RR distance 

The input should be a beat array (70 samples) plus three mentioned features. The output is a label for that beat.  


In [10]:
path = '../input/aibiomed/training_set/'

In [11]:
from scipy.signal import resample

def downsample(signal_orig, rpeaks, signal_fs, fs_new=128):
    
    """ Given the signal and its annotation, downsample the signal and 
        modify the positions of the annotations 
    """
    fs = signal_fs
    
    new_num_samples = int(len(signal_orig)/fs*fs_new)
    downsamp_signal = np.array(resample(signal_orig, new_num_samples))
    
    downsamp_ann = rpeaks/(fs/fs_new)
    downsamp_ann = np.round(downsamp_ann).astype(int)

    return downsamp_signal, downsamp_ann

In [12]:
def calculate_signal_properties(signal_orig, peaks):
    rr_dif = []
    
    for i in range(len(peaks)-1):
        rr_dif_temp = peaks[i+1]-peaks[i]
        rr_dif.append(rr_dif_temp)
        
    #   mean
    mean_RR = np.mean(rr_dif)
    mean_Peaks = np.mean(signal) # Should be 0
    
    #   median
    median_RR = np.median(rr_dif)
    median_Peaks = np.median(signal)

    #   standard deviation
    std_RR = np.std(rr_dif)
    std_Peaks = np.std(signal)

    #   variance
    # var_RR = stats.tvar(rr_dif)
    # var_Peaks = stats.tvar(signal)
    
    # #   Skewness
    # skew_RR = stats.skew(rr_dif)
    # skew_Peaks = stats.skew(signal)

    # #   Kurtosis
    # kurt_RR = stats.kurtosis(rr_dif)
    # kurt_Peaks = stats.kurtosis(signal, axis=0, fisher=True, bias=True)

    # features = np.hstack([mean_RR, mean_Peaks, median_RR, median_Peaks, std_RR, std_Peaks, 
    #                       var_RR, var_Peaks, skew_RR, skew_Peaks, kurt_RR, kurt_Peaks])

    features = np.hstack([mean_RR, mean_Peaks, median_RR, median_Peaks, std_RR, std_Peaks])

    return features

In [13]:
output_path = './'
dataset = []
labels = []
sample_names = []
lead_num = []
r_peaks_position = []
beat_names = []
data_path = output_path + 'data'
labels_path = output_path + 'labels'
signal_features = []

for _, _, filenames in os.walk(path):    
    for filename in filenames:
        
        if filename[-7:-4]=='rpk' or filename[-7:-4]=='ann':
            continue
        
        # Read three files for each signal 
        mat = scipy.io.loadmat(os.path.join(path,filename))
        mat_rpeaks = scipy.io.loadmat(os.path.join(path,filename[:-4]+'_rpk.mat'))
        mat_ann = scipy.io.loadmat(os.path.join(path,filename[:-4]+'_ann.mat'))
        
        # print(mat)
        signal = mat['ecg']
        rpeaks = mat_rpeaks['rpeaks']
        annots = mat_ann['labels'] 
                    
        D = 35              # Distance around R peak

        # ECG Segmentation and Beat Processing
        for lead in [0, 1]:
            signal_lead = signal[:,lead]
            lead_temp = lead
            feature_temp = calculate_signal_properties(signal_lead, rpeaks)
            
             # Resample the signals to 128Hz
            fs = int(filename[-7:-4])
            if fs == 250:
                signal_lead, rpeaks = downsample(signal_lead, rpeaks, fs)

            # Filter the signals
            signal_lead = filter_signal(signal_lead, fs=128)

            for i, position in enumerate(rpeaks):
                position = position[0]

                # Check the boundaries
                if position-D < 0:
                    beat = signal_lead[:2*D]   
                elif position+D > len(signal_lead)-1:
                    beat = signal_lead[-2*D:]
                else:
                    beat = signal_lead[position-D:position+D] 

                # Normalize the beat between [-1,1]
                beat = (2 *(beat - min(beat))/(max(beat)-min(beat))) - 1

                # Lists of beats (array) and labels 
                dataset.append(beat)
                labels.append(annots[i])
                
                # Save all beats and their annots as individual files
                beat_name = filename[:-4] + '_L' + str(lead) + '_Beat_' + str(i+1)
                
                sample_names.append(filename)
                beat_names.append(beat_name)
                r_peaks_position.append(rpeaks)
                lead_num.append(lead_temp)
                signal_features.append(feature_temp)
    

In [14]:
# signal_features

In [15]:
# DataFrame with raw signals
df_dataset = pd.DataFrame(list(map(np.ravel, dataset)))
df_labels = pd.DataFrame(labels, columns=['output_label'])

# df_raw = pd.concat([df_dataset, df_labels], axis = 1)
# df_raw

In [16]:
# TODO DataFrame with signal + hand-crafted features
# df_signal_properties = pd.DataFrame(signal_features, columns=['mean_RR','mean_Peaks','median_RR','median_Peaks','std_RR','std_Peaks',
#                         'var_RR','var_Peaks','skew_RR','skew_Peaks','kurt_RR','kurt_Peaks'])

df_signal_properties = pd.DataFrame(signal_features, columns=['mean_RR','mean_Peaks','median_RR','median_Peaks','std_RR','std_Peaks'])
df_beat_names = pd.DataFrame(beat_names, columns=['beat_name'])
df_lead = pd.DataFrame(lead_num, columns=['lead_num'])
# df_mixed = pd.concat([df_beat_names, df_lead, df_dataset, df_signal_properties, df_labels], axis = 1)
# df_mixed


In [17]:
# df_mixed = df_mixed.drop(columns=['beat_name'])

In [18]:
X = pd.concat([df_lead, df_dataset, df_signal_properties], axis = 1)
y = df_labels

In [19]:
# from sklearn.model_selection import train_test_split
# # import matplotlib.pyplot as plt

# train, test = train_test_split(df_mixed, test_size=0.2, random_state=seed,
#                                shuffle=True, stratify=df_raw['output_label'])
# train_labels = train['output_label']
# train_features = train.drop(columns='output_label')

# test_labels = test['output_label']
# test_features = test.drop(columns='output_label')


In [20]:
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

def plot_confusionMatrix(test_labels, test_predicted, clf):
    cm = confusion_matrix(test_labels, test_predicted, labels=clf.classes_)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                                display_labels=['N', 'S', 'V'])
    disp.plot()
    plt.show()
    return

def testModel(train_features, train_labels, test_features, test_labels, clf):
    clf.fit(train_features, train_labels)

    # Predict test labels
    test_predicted = clf.predict(test_features)

    # Plot confusion matrix
    plot_confusionMatrix(test_labels, test_predicted, clf)

    # Display classification results
    print(classification_report(test_labels, test_predicted, target_names=['N', 'S', 'V']))

In [21]:
from sklearn.model_selection import train_test_split

test_size=0.2 
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=test_size, random_state=seed)

In [23]:
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE

NORMAL = "N"
VENTRICULAR = "V"
SUPER_VENTRICULAR = "S"

# Under and oversampling to balance the unbalanced dataset
under = RandomUnderSampler(sampling_strategy={NORMAL: 100000})
X_res, y_res = under.fit_resample(Xtrain, ytrain)
smote = SMOTE(sampling_strategy={VENTRICULAR: 100000, SUPER_VENTRICULAR: 100000})
X_res, y_res = smote.fit_resample(X_res, y_res)

In [43]:
# from tensorflow.keras.utils import to_categorical
# from sklearn.preprocessing import OneHotEncoder

# y_res = pd.get_dummies(y_res)
# ytest = pd.get_dummies(ytest)


In [ ]:
param_grid = [
              {'n_estimators': [100, 200, 50], 
               'criterion': ['gini', 'entropy'],
               'max_depth': [None, '3']}
              ]

# 2. Crossvalidate model to obtain the best fit 
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.ensemble import RandomForestClassifier


cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
clf = RandomizedSearchCV(RandomForestClassifier(), param_grid, cv=cv, scoring='accuracy')
clf.fit(X_res, y_res)

for param, score in zip(clf.cv_results_['params'], clf.cv_results_['mean_test_score']):
    print(param, score)

print('Best combination of hyperparameters: ' + str(clf.best_params_))

clf_best = RandomForestClassifier(**clf.best_params_)
testModel(Xtrain, ytrain, Xtest, ytest, clf_best)

In [ ]:
# from sklearn.ensemble import RandomForestClassifier

# clf = RandomForestClassifier(n_estimators=100,
#                        criterion='gini', 
#                        max_depth=None, 
#                        min_samples_split=2, 
#                        min_samples_leaf=1,
#                        min_weight_fraction_leaf=0.0,
#                        max_features='auto',
#                        max_leaf_nodes=None,
#                        min_impurity_decrease=0.0,
#                        bootstrap=True,
#                        oob_score=False,
#                        n_jobs=None,
#                        random_state=seed, 
#                        verbose=0,
#                        warm_start=False,
#                        class_weight=None,
#                        ccp_alpha=0.0, 
#                        max_samples=None)

# testModel(X_res, y_res, Xtest, ytest, clf)

In [ ]:
# # feed-forward neural network
# def build_ffnn(input_shape):

#     # Build the neural network layer by layer
#     input_layer = tf.keras.layers.Input(shape=input_shape, name='Input')
#     hidden_layer1 = tf.keras.layers.Dense(units=128, activation='relu', name='Hidden1')(input_layer)
#     hidden_layer2 = tf.keras.layers.Dense(units=64, activation='relu', name='Hidden2')(hidden_layer1)
#     flatten = tf.keras.layers.GlobalMaxPool1D()(hidden_layer2)
#     output_layer = tf.keras.layers.Dense(units=num_classes, activation='sigmoid', name='Output')(flatten)

#     # Connect input and output through the Model class
#     model = Model(inputs=input_layer, outputs=output_layer, name='FFNN')

#     # Compile the model
#     loss = 'categorical_crossentropy'
#     learning_rate = 0.02
#     optimizer = keras.optimizers.Adam(learning_rate)
#     model.compile(loss=loss, optimizer=optimizer, metrics = 'accuracy')

#     # Return the model
#     return model


In [ ]:
# inputshape = len(X_res)
# FFN = build_ffnn(inputshape)
# FFN.summary()

In [44]:
# Xshape = X.shape
# print(Xshape)
# num_classes = 3

In [55]:
# FFN = keras.Sequential([
#     keras.layers.Input((1, 77)),
#     keras.layers.Dense(128, activation=tf.nn.relu),
#     keras.layers.Dense(64, activation=tf.nn.relu),
#     keras.layers.Flatten(),
#     keras.layers.Dense(num_classes, activation=tf.nn.sigmoid)
# ])

In [56]:
# learning_rate = 0.01
# FFN.compile(
#     optimizer = keras.optimizers.Adam(learning_rate),
#     loss = 'categorical_crossentropy',
#     metrics = ['accuracy']
# )

In [57]:
# history = FFN.fit(
#     x = X_res,
#     y = y_res, 
#     validation_data=(Xtest, ytest),
#     batch_size = 64,
#     epochs = 10
# ).history

In [48]:
# plt.figure(figsize = (12,4))
# plt.subplot(1,2,1)
# plt.plot(history['loss'], label="training loss")
# plt.plot(history['val_loss'], label="validation loss")
# plt.title('Lossfunction Sequential FFN:')
# plt.legend()

# plt.subplot(1,2,2)
# plt.plot(history['accuracy'], label="training accuracy")
# plt.plot(history['val_accuracy'], label="validation accuracy")
# plt.title('Accuracy Sequential FFN:')
# plt.legend()